# 1. Documentation

## Version 1:
- Program base (voice activation and audio response)

## Libraries

- SpeechRecognition: to recognize audio input
- PyAudio: for audio I/O
- Requests: to send http requests
- Playsound: to play audio files (not necessary)
- gTTS: for voice recognition and audio response on the robot's side
- IMDbPy: for imdb search queries
- json: to parse json files

In [94]:
import speech_recognition as sr
from IPython.display import *
from gtts import gTTS
import os
from playsound import playsound
import datetime
from datetime import date
import webbrowser
import time
import json
import requests
import imdb
import random
import urllib3

In [95]:
urllib3.disable_warnings()

if os.path.isfile('ready.mp3'):
    os.remove("ready.mp3")

if os.path.isfile('response.mp3'):
    os.remove("response.mp3")
if os.path.isfile('hello.mp3'):
    os.remove("hello.mp3")

# 2. Functions

### Basic functions

In [96]:
def error_message():
    tts = gTTS(text='Ich habe Sie nicht richtig verstanden. Beginnen wir noch mal von vorne.', lang='de')
    # check if the file already exists, if so just play it (if not save it and then play it)
    if os.path.isfile('error.mp3'):
        playsound("error.mp3")
    else:
        tts.save("error.mp3")
        playsound("error.mp3")
    
    print("ERROR: Ihr Audio-Guide ist verwirrt!")

In [97]:
def welcome():
    tts = gTTS(text='Herzlich willkommen! Mein Name ist Alex und ich bin Ihr Audio-Guide. In kürze können Sie sich über die jeweiligen Stationen informieren.', lang='de')
    # check if the file already exists, if so just play it (if not save it and then play it)
    if os.path.isfile('hello.mp3'):
        playsound("hello.mp3")
    else:
        tts.save("hello.mp3")
        playsound("hello.mp3")

In [98]:
def ready():
    tts = gTTS(text='Wie kann ich Ihnen behilflich sein? Über welche Station hätten Sie gerne Informationen?', lang='de')
    # check if the file already exists, if so just play it (if not save it and then play it)
    if os.path.isfile('ready.mp3'):
        playsound("ready.mp3")
    else:
        tts.save("ready.mp3")
        playsound("ready.mp3")

### Intent pool
The program looks at the json file to check if a voice command fits the programs intents

In [99]:
def intents(req):
    # debugging:
    print("DEBUG 6: In intents function")
    # can only play one response - maybe deleting the file again and creating it when needed will help
    print("DEBUG 7: Deleting previous responses")
    if os.path.isfile('resonse.mp3'):
        os.remove("response.mp3")
        
    # open json file
    with open('intents.json') as json_file:
        # load json file
        data = json.load(json_file)

        # debugging:
        print("DEBUG 8: Looking through intents in the json file")
        
        # go through intents
        for i in data['intents']:
            # find the right request
            for p in i['patterns']:
                if req == p.lower():
                    # debugging:
                    print("DEBUG 9: Request matches intent. Looking for response")
                    
                    # pick a randomized response for that request
                    count = 0
                    for resp in i['responses']:
                        count += 1
                        
                    index = random.randint(0, count-1)
                    
                    # debugging:
                    print("DEBUG 10: Found a response. About to send it to tts")
                    
                    response = i['responses'][index]
                    tts = gTTS(response, lang='de')
                    
                    # debugging:
                    print("DEBUG 11: This is the response i should give the user")
                    print(response)
                    
                    ##################################################################################################
                    ## PROBLEM: ##
                    # I can save the first time after a request but not the 2nd time around. The previous response.mp3 
                    # file should have been deleted so that can't be the issue
                    ##################################################################################################
                    
                    tts.save("response.mp3")
                    playsound("response.mp3")
                    
                    # can only play one response - maybe deleting the file again and creating it when needed will help
                    print("DEBUG 12: Deleting previous responses again")
                    if os.path.isfile('resonse.mp3'):
                        os.remove("response.mp3")
                    
                    # if the user wishes to exit the program
                    if response == "Auf Wiedersehen! Ich hoffe ich konnte Ihnen behilflich sein." or response == "Ich hoffe Sie haben noch einen wundervollen Tag!":
                        tts = gTTS(response, lang='de')
                        tts.save("response.mp3")
                        playsound("response.mp3")
                        return False
        
        
        # TO-DO: Respond when the request doesn't fit without breaking the code :))))
        
        
        # tts = gTTS('Verzeihung, aber ich verstehe Ihre Anfrage nicht. Probieren wir es nochmals von Anfang an!', lang='de')
                    
        # can only play one response - maybe deleting the file again and creating it when needed will help
        # print("DEBUG 13: Deleting previous responses")
        # if os.path.isfile('resonse.mp3'):
            # os.remove("response.mp3")
                        
        # tts.save("response.mp3")
        # playsound("response.mp3")
        
        return True

        

# 3. Speech recognition

### Audio/input parser
Record audio and send it to Google's API to parse/"translate"

In [100]:
#make a recognizer query while checking for different errors and return an according response 
def mic_recognition(recognizer, microphone):
    if not isinstance(recognizer, sr.Recognizer):
        raise TypeError("`recognizer` must be `Recognizer` instance")

    if not isinstance(microphone, sr.Microphone):
        raise TypeError("`microphone` must be `Microphone` instance")
        
    response = {
      "response_trans": None,
      "error": None
    }
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source, duration=0.5)
        audio_recorded = recognizer.listen(source)
    try:
        response["response_trans"] = recognizer.recognize_google(audio_recorded, language = "de-DE")
    except sr.RequestError:
        #API was unreachable or unresponsive
        response["error"] = "ERROR: Die API ist nicht verfügbar."
    except sr.UnknownValueError:
        #speech was unintelligible
        response["error"] = "ERROR: Der Audio-Guide konnte den Sprach-Input nicht verstehen."
        
    return response

### Speech recognition
Immediate interaction with the robot takes place here. The program takes the received input and checks if said input can be found in its intent pool

In [101]:
def activate_speechrec():
    # debugging:
    print("DEBUG 3: In activate_speechrec. Sending welcome message")
    
    welcome()
    talking = True
    while(talking):
        ready()
        
        # debugging:
        print("DEBUG 4: In activate_speechrec while loop. Ready for request.")
        
        request = mic_recognition(rec, mic)
        if request["error"] is None:
            
            # debugging:
            print("DEBUG 5: Got a request. Going into intents function")
            
            retval = intents(request["response_trans"].lower())
            if retval == False:
                continue
        else:
            print(request["error"])
            error_message()

### Voice activation
Activating robot interaction starts with the phrase "hallo alex"

In [102]:
keyword = "hallo alex"
success = False

#Initialize recognizer instance and a microphone as a source
rec = sr.Recognizer()
mic = sr.Microphone()
#try 2 times to get the correct keyword
#for i in range (2):

#could say while(1) to listen endlessly until notebook is closed?
while(1):
    request = mic_recognition(rec, mic)
#check if voice was recognized
    if request["error"] is None:
        if request["response_trans"].lower() == keyword:
            success = True
            
            # debugging:
            print("DEBUG 1: Recognized keyword")
            break

#now you can speak to the bot
if success is True:
    
    # debugging:
    print("DEBUG 2: Entering activate_speechrec function")
    activate_speechrec()
else:
    print(request["error"])
    error_message()

DEBUG 1: Recognized keyword
DEBUG 2: Entering activate_speechrec function
DEBUG 3: In activate_speechrec. Sending welcome message
DEBUG 4: In activate_speechrec while loop. Ready for request.
DEBUG 5: Got a request. Going into intents function
DEBUG 6: In intents function
DEBUG 7: Deleting previous responses
DEBUG 8: Looking through intents in the json file
DEBUG 9: Request matches intent. Looking for response
DEBUG 10: Found a response. About to send it to tts
DEBUG 11: This is the response i should give the user
Station Nummer drei kann ebenfalls etwas machen. Ich weiss zwar immer noch nicht was, aber es ist etwas!
DEBUG 12: Deleting previous responses again
DEBUG 4: In activate_speechrec while loop. Ready for request.
DEBUG 5: Got a request. Going into intents function
DEBUG 6: In intents function
DEBUG 7: Deleting previous responses
DEBUG 8: Looking through intents in the json file
DEBUG 9: Request matches intent. Looking for response
DEBUG 10: Found a response. About to send it to

PermissionError: [Errno 13] Permission denied: 'response.mp3'